In [115]:
import json, re
from scripts.bugsinpy_utils import *
from datetime import datetime
import time
import os

from dotenv import load_dotenv

load_dotenv()

In [116]:
# from ollama import Client
# client = Client(host="https://dirty-tables-mix.loca.lt")

# from openai import OpenAI
# client = OpenAI(api_key=os.environ["OPENAI_KEY"])
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENROUTER_KEY"], base_url="https://openrouter.ai/api/v1")


In [117]:
SYSTEM_PROMPT = """Given a stack trace, code snippet, and optionally a test file, identify the root cause of the error and output the fully corrected version of the entire function always, regardless of how much code changes.
- Don't reason too much, be concise.
- Analyze the stack trace and connect it to errors in the code snippet.
- Consider assertions, inputs, and behavior from the test file if provided.
- Reason step-by-step about the possible fix; document this step-by-step reasoning in your output.
- After thoroughly understanding and explaining your reasoning, summarize why you believe your fix is appropriate.
- Do not use any new imports than what is already given to you.
- Each changed function needs its own ```python tags, do not put 2 functions in the same tag.
- Always output the entire function source code with the fix applied, matching the original style and programming language. Do not change the function name.
- Format your output code using ```python tags; only the function code should be inside these tags. If there's multiple functions, put them in separate ```python tags
"""

MODEL_NAME = "all_deepseek-r1-0528"
FIRST_TIME=False
PREVIOUS_RESULTS_PATH="tmp/ast/results/llm/single/all_deepseek-r1-0528_07_27_2025__20_20_36.json"

In [118]:
def generate_code(trace_back, chunks, previous_chat=None):
    if previous_chat is not None:
        previous_chat.append({"role": "user", "content": f"New error: {trace_back}"})
    else:
        previous_chat = [
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": f"Traceback: {trace_back}",
            },
            {
                "role": "user",
                "content": "Code: " + "\n".join(chunks),
            },
        ]
    # response = client.chat(
    #     # model=MODEL_NAME,
    #     model="phi4-reasoning:14b-plus-q4_K_M",
    #     messages=previous_chat
    # )
    # previous_chat.append({
    #     "role": "assistant",
    #     "content": response["message"]["content"]
    # })
    # print(previous_chat[-1])
    
    
    # response = client.responses.create(
    #     model="o4-mini",
    #     input=[
    #         {
    #             "role": "system",
    #             "content": [{"type": "input_text", "text": SYSTEM_PROMPT}],
    #         },
    #         {
    #             "role": "user",
    #             "content": [{"type": "input_text", "text": f"Traceback: {trace_back}"}],
    #         },
    #         {
    #             "role": "user",
    #             "content": [
    #                 {"type": "input_text", "text": "Code: " + "\n".join(chunks)}
    #             ],
    #         },
    #     ],
    #     text={"format": {"type": "text"}},
    #     reasoning={
    #         "effort": "medium"
    #     },
    #     tools=[],
    #     temperature=1,
    #     max_output_tokens=16384,
    #     top_p=1,
    #     store=True,
    # )
    # return response.output_text
    
    completion = client.chat.completions.create(
        extra_body={}, model="deepseek/deepseek-r1-0528", messages=previous_chat
    )
    print(completion.choices)
    previous_chat.append({
        "role": "assistant",
        "content": completion.choices[0].message.content
    })
    
    
    return previous_chat


def extract_code(text: str):
    # Extract fenced code blocks first
    fence_pattern = r"```(?:([a-zA-Z]+)\n)?(.*?)```"
    code_blocks = re.findall(fence_pattern, text, re.DOTALL)

    def_pattern = r"(def\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\(.*?(?:(?=^def\s)|\Z))"
    func_codes = []
    func_names = []

    for lang, block in code_blocks:
        if lang != "python":
            continue
        # Extract individual functions from within the block
        matches = re.findall(def_pattern, block, re.DOTALL | re.MULTILINE)
        for match in matches:
            func_codes.append(match.strip())
            # Extract just the function name
            name_match = re.match(r"def\s+([a-zA-Z_][a-zA-Z0-9_]*)", match)
            if name_match:
                func_names.append(name_match.group(1))

    return func_codes, func_names

In [119]:
llm_results = []

In [120]:
bugs = get_bugs("youtube-dl")
bugs = sorted(bugs)
# bugs = ["1", "2", "3", "5", "6"]
passed_bugs = [
    "1",
    "2",
    "3",
    "8",
    "9",
    "11",
    "12",
    "16",
    "17",
    "18",
    "19",
    "22",
    "24",
    "27",
    "29",
    "30",
    "31",
    "32",
    "33",
    "35",
    "36",
    "38",
    "39",
]
for bug in bugs:
    trace_back = get_raw_traceback("youtube-dl", bug)
    bug_info = get_bug_info("youtube-dl", bug)
    buggy_commit_id = bug_info["buggy_commit_id"]
    checkout_to_commit("youtube-dl", buggy_commit_id)
    changed_file = parse_changed_files("youtube-dl", bug)
    function_names = parse_changed_function_names("youtube-dl", bug)
    if len(changed_file) > 1:
        continue

    changed_file = changed_file[0]
    print(bug, function_names)

    flag = False
    for result in llm_results:
        if result["bug"] == bug:
            flag = True
            break

    if flag:
        print("Skipped")
        continue
    chunks = {}
    test_functions = extract_function_name_from_traceback(trace_back)
    for test_function in test_functions:
        tree, sources = test_to_source_code(
            "youtube-dl", bug_info["test_file"], test_function, max_depth=1
        )
        for (file_path, func_name), src in sources.items():
            chunks[func_name] = src

    if bug in passed_bugs:
        with open(PREVIOUS_RESULTS_PATH, "r", encoding="utf-8") as f:
            data = json.load(f)
            for x in data:
                if x["bug"] == bug:
                    data_bug = x
                    break

            llm_results.append(data_bug)
            print(f"Bug_id: {bug} is done")
            continue
    # if not FIRST_TIME:
    #     with open(PREVIOUS_RESULTS_PATH, "r", encoding="utf-8") as f:
    #         data = json.load(f)
    #         data_bug = data[int(bug) - 1]
    #         for name, src in chunks.items():
    #             if name in data_bug["changes_function_class_names"]:
    #                 chunks[name] = data_bug["change"][data_bug["changes_function_class_names"].index(name)]
    #                 print("Updated the code with previous")
    if not FIRST_TIME:
        with open(PREVIOUS_RESULTS_PATH, "r", encoding="utf-8") as f:
            data = json.load(f)

            for x in data:
                if x["bug"] == bug:
                    data_bug = x
                    break

            chats = generate_code(
                trace_back, list(chunks.values()), data_bug["previous_chat"]
            )
    else:
        chats = generate_code(trace_back, list(chunks.values()))

    codes, changed_function_names = extract_code(chats[-1]["content"])
    llm_results.append(
        {
            "project": "youtube-dl",
            "bug": bug,
            "file": changed_file,
            "actual_function": function_names,
            "given_functions": list(chunks.keys()),
            "previous_chat": chats,
            "changes_function_class_names": changed_function_names,
            "change": codes,
        }
    )

    print(f"Bug_id: {bug} is done")

HEAD is now at 99036a129 [pornflip] Relax _VALID_URL (closes #16258)


1 ['_match_one']


KeyboardInterrupt: 

In [ ]:
llm_results 

[{'project': 'youtube-dl',
  'bug': '1',
  'file': 'youtube_dl/utils.py',
  'actual_function': ['_match_one'],
  'given_functions': ['test_match_str',
   'match_str',
   '_match_one',
   'parse_filesize',
   'lookup_unit_table'],
  'previous_chat': [{'role': 'system',
    'content': "Given a stack trace, code snippet, and optionally a test file, identify the root cause of the error and output the fully corrected version of the entire function always, regardless of how much code changes.\n\n- Analyze the stack trace and connect it to errors in the code snippet.\n- Consider assertions, inputs, and behavior from the test file if provided.\n- Reason step-by-step about the possible fix; document this step-by-step reasoning in your output.\n- After thoroughly understanding and explaining your reasoning, summarize why you believe your fix is appropriate.\n- Do not use any new imports than what is already given to you.\n- Each changed function needs its own ```python tags, do not put 2 functio

In [ ]:
now = datetime.now()

os.makedirs("tmp/ast/results/llm/single", exist_ok=True)
with open(f"tmp/ast/results/llm/single/{MODEL_NAME}_{now.strftime('%m_%d_%Y__%H_%M_%S')}.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(llm_results, indent=2))